In [1]:
### import required lib ###
from keras.models import Sequential
from keras.layers import Dense, Activation
import pandas as pd

C:\Users\rnt1013\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
### read dataset from disk ###
data = pd.read_csv('data.csv')
data.columns = ['DATE', 'SMA', 'EMA', 'MOM', 'STOCK', 'STOCD', 'RSI', 'MACD', 'LWR', 'ADO', 'CCI', 'LABEL']
sensex_data = pd.read_csv('SENSEX.csv')
sensex_data.columns = ['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE']

In [3]:
### merge data to get 'CLOSE' column in the data frame ###
sensex_data = sensex_data.iloc[19:,:].reset_index(drop = True).copy()
required_sensex_data = sensex_data[['DATE', 'CLOSE']]
required_sensex_data = required_sensex_data.merge(data, on='DATE', how='right')
# required_sensex_data.head()

In [4]:
### Feature and Label for model  ###
label = required_sensex_data['CLOSE']
feature = required_sensex_data.drop(['CLOSE', 'DATE', 'LABEL'], axis=1)

# print(label.head())
# print(feature.head())

In [5]:
### train-test split
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(feature, label, test_size=0.2, random_state=None, shuffle=False)

# print(train_x)
# print(train_y)
# print(test_x)
# print(test_y)

**ERROR Function**

In [10]:
### Error ###
def error_function(test_y, predicted_y):
    import numpy as np
    MAPE = (1/len(test_y)) * np.sum((np.absolute(test_y - predicted_y) / np.absolute(test_y)) * 100)
    MAE = np.sum(np.absolute(test_y - predicted_y) / np.absolute(test_y)) / len(test_y)
    rRMSE = np.sqrt(np.sum((test_y - predicted_y) ** 2) / len(test_y))
    MSE = np.sum((test_y - predicted_y) ** 2) / len(test_y) 
    return MAPE, MAE, rRMSE, MSE

**SRV model** 

In [11]:
def srv_model(train_x, train_y, test_x):
    from sklearn.svm import SVR
    import numpy as np

    clf = SVR(C=1.0, epsilon=0.2, gamma=4.0)
    clf.fit(train_x, train_y)
    predicted_y = clf.predict(test_x)
    return predicted_y

In [12]:
### final output for single stage - SVR ###
svr_predicted_y = srv_model(train_x, train_y, test_x)
svr_MAPE, svr_MAE, svr_rRMSE, svr_MSE = error_function(test_y, svr_predicted_y)
print(svr_MAPE, svr_MAE, svr_rRMSE, svr_MSE)

35.55841724104483 0.3555841724104483 6423.300647194327 41258791.20424706


**ANN Model**

In [23]:
def ann_model(train_x, train_y, test_x):
    model = Sequential()
    model.add(Dense(64, activation='tanh', kernel_initializer='normal', input_dim=10))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(optimizer='adagrad', loss='mean_squared_error')

    model.fit(train_x, train_y,
              epochs=3000,
              batch_size=128)
    ann_predicted_y = model.predict(test_x)
    ann_predicted_y = ann_predicted_y.reshape(-1)
    return ann_predicted_y

In [24]:
### final output for single stage - ANN ###
ann_predicted_y = ann_model(train_x, train_y, test_x)
ann_MAPE, ann_MAE, ann_rRMSE, ann_MSE = error_function(test_y, ann_predicted_y)
print(ann_MAPE, ann_MAE, ann_rRMSE, ann_MSE)

Epoch 1/20
1989/1989 [==============================] - 0s 111us/step - loss: 154086513.6410
Epoch 2/20
1989/1989 [==============================] - 0s 20us/step - loss: 154066490.3449
Epoch 3/20
1989/1989 [==============================] - 0s 24us/step - loss: 154046064.6596
Epoch 4/20
1989/1989 [==============================] - 0s 20us/step - loss: 154026139.2298
Epoch 5/20
1989/1989 [==============================] - 0s 20us/step - loss: 154006754.9120
Epoch 6/20
1989/1989 [==============================] - 0s 18us/step - loss: 153987913.9588
Epoch 7/20
1989/1989 [==============================] - 0s 18us/step - loss: 153969414.0493
Epoch 8/20
1989/1989 [==============================] - 0s 22us/step - loss: 153951242.6506
Epoch 9/20
1989/1989 [==============================] - 0s 22us/step - loss: 153933383.4731
Epoch 10/20
1989/1989 [==============================] - 0s 24us/step - loss: 153915962.1197
Epoch 11/20
1989/1989 [==============================] - 0s 22us/step - loss: 

**RANDOM FOREST Model**

In [27]:
def random_forest_reg(train_x, train_y, test_x):
    from sklearn.ensemble import RandomForestRegressor
    import numpy as np

    rfr = RandomForestRegressor(max_depth=2, random_state=0)
    rfr.fit(train_x, train_y)
    rfr_predicted_y = rfr.predict(test_x)
    return rfr_predicted_y

In [28]:
### final output for single stage - SVR ###
svr_predicted_y = random_forest_reg(train_x, train_y, test_x)
rfr_MAPE, rfr_MAE, rfr_rRMSE, rfr_MSE = error_function(test_y, svr_predicted_y)
print(rfr_MAPE, rfr_MAE, rfr_rRMSE, rfr_MSE)

5.038216014701784 0.050382160147017845 1024.4081932184984 1049412.146333188


**Two stage model**

In [ ]:
# just call function for two stage model